In [ ]:
!pip uninstall transformers -y
!pip install --upgrade --no-cache-dir -q transformers==4.52.1

In [ ]:
import transformers
from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
import requests
import copy
%matplotlib inline  
print(transformers.__version__)

In [ ]:
import gc
llm_model=None
vision_processor=None
vision_model=None
llm_tokenizer=None
gc.collect()
torch.cuda.empty_cache()


In [ ]:
model_id = 'microsoft/Florence-2-large-ft'
vision_model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, revision="main", torch_dtype='auto').eval().cuda()
vision_processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True, revision="main")


In [ ]:
help(vision_model.generate)

# Блок генерации Bearth признаков
# главное это encoder_hidden_states[-1]

In [ ]:
import torch

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)
task_prompts = [  
    'Describe in detail what is shown in the image.',  
    'What is the text in the image?',  
    'Locate the objects in the image, with their descriptions.'  
]  
  
   
images = [image] * 3   
inputs = vision_processor(text=task_prompts, images=images, return_tensors="pt", padding=True).to('cuda', torch.float16)
   
generated_ids = vision_model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      return_dict_in_generate=True,
      output_hidden_states=True,
      num_beams=3,
    )

last_hidden_state = generated_ids.encoder_hidden_states[-1]
print(last_hidden_state.shape)


In [ ]:
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)
task_prompt = '<CAPTION>'


In [ ]:
 vision_model.vision_tower

In [ ]:
print(dir(vision_model.vision_tower))


# Блок генерации Depth признаков
# главное это vision_model.vision_tower.forward_features_unpool

In [ ]:

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)
task_prompts = [  
    'Describe in detail what is shown in the image.',  
    'What is the text in the image?',  
    'Locate the objects in the image, with their descriptions.'  
]  
  
   
     
inputs = vision_processor(text=task_prompts, images=image, return_tensors="pt", padding=True)    
        
inputs["input_ids"] = inputs["input_ids"].to('cuda')    
inputs["pixel_values"] = inputs["pixel_values"].to('cuda', dtype=torch.float16)   

In [ ]:

with torch.no_grad():
    image_features = vision_model.vision_tower.forward_features_unpool(inputs["pixel_values"])
print(image_features.shape)